In [1]:
import os
import argparse
import pandas as pd
import numpy as np
import pickle
from utils_test import *

In [2]:
from jellyfish._jellyfish import damerau_levenshtein_distance
import distance
from sklearn import metrics

In [3]:
from keras.models import load_model

Using Theano backend.


In [4]:
name = 'helpdesk'
args = {
    'inputdir': '../input/{}/'.format(name),   
    'outputdir': './output_files/{}/'.format(name),
    'modelname': 'model_00-1.75.h5',
}

args = argparse.Namespace(**args)

In [5]:
test = pd.read_csv(args.inputdir+'test.csv')
test = transformDf(test)

In [6]:
with open(args.inputdir+'parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [7]:
with open(args.inputdir+'preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

# Utils

In [8]:
model = load_model(args.outputdir+args.modelname)

OSError: Unable to open file (Unable to open file: name = './output_files/helpdesk/model_00-1.75.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [13]:
test_groupByCase = test.groupby(['CaseID'])

#get features all data
sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4 = getFeature(test_groupByCase)

#get output all data
next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = getOutput(test_groupByCase)

# Check longest case --> suffix: 2 to 7
test_len = findLongestLength(test_groupByCase)
test_len - 5

8

In [11]:
def get_remaining_trace_accuracy(true_labels, pred_probabilities, targetchartoindice):
    true_trace, pred_trace = getRemaingTrace(true_labels, pred_probabilities, targetchartoindice)
    count=0.0
    for i in range(len(true_trace)):
        if true_trace[i] == pred_trace[i]:
            count+=1
    acc = count/len(true_trace)
    print('Accuracy of remaining trace: {}%'.format(acc*100))

In [12]:
def evalAct(true_label, pred_prob, targetchartoindice):
    #for log-loss: get probabilities
    gt_a = one_hot_encode(true_label, targetchartoindice)
    log_loss = metrics.log_loss(gt_a, pred_prob)
    print('Log-loss: {}'.format(log_loss))
    
    #for accuracy: get labels
    gt_a_label = true_label
    pred_a_label = getLabel(pred_prob, targetchartoindice)
    acc = metrics.accuracy_score(gt_a_label, pred_a_label)
    print('Accuracy: {}%'.format(acc*100))
    acc_3 = get_top3_accuracy(pred_prob, gt_a_label, targetchartoindice)
    print('Top 3 accuracy: {}%'.format(acc_3*100))

In [13]:
def evalTime(true_time, pred_time, divisor):
    #get time
    gt_t = true_time
    pred_t = inverseTime(pred_time, divisor)
    
    mse = metrics.mean_squared_error(gt_t, pred_t)
    mae = metrics.mean_absolute_error(gt_t, pred_t)
    median = metrics.median_absolute_error(gt_t, pred_t)
    
    print('Mean Squared Error: {0}s\t| {1} days'.format(mse, mse/86400))
    print('Mean Absolute Error: {0}s\t| {1} days'.format(mae, mae/86400))
    print('Median Absolute Error: {0}s\t| {1} days'.format(median, median/86400))

In [14]:
# DL distance
#distance.nlevenshtein(gt_a_label, pred_a_label) #0.1731066460587326

#damerau_levenshtein_distance(gt_a_label, pred_a_label) #784

#distance.jaccard(gt_a_label, pred_a_label) #0.4444444444444444

In [15]:
def getSuffix(suffix, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t):
    sentences_2 = []
    sentences_t_2 = []
    sentences_t2_2 = []
    sentences_t3_2 = []
    sentences_t4_2 = []
    next_chars_2 = []
    next_chars_t_2 = []

    for i in range(len(sentences)):
        if len(sentences[i]) == suffix:
            sentences_2.append(sentences[i])
            sentences_t_2.append(sentences_t[i])
            sentences_t2_2.append(sentences_t2[i])
            sentences_t3_2.append(sentences_t3[i])
            sentences_t4_2.append(sentences_t4[i])
            next_chars_2.append(next_chars[i])
            next_chars_t_2.append(next_chars_t[i])
    return sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2

# All data

## Predict

In [20]:
pred = model.predict(X_test, verbose=0)
pred_a = pred[0]
pred_t = pred[1]

In [21]:
pred[0].shape, pred[1].shape

((4529, 10), (4529, 1))

## Evaluate

### Activity

In [ ]:
get_remaining_trace_accuracy(next_chars, pred_a, targetchartoindice)

In [ ]:
evalAct(next_chars, pred[0], targetchartoindice)

### Time

In [ ]:
evalTime(next_chars_t, pred_t, divisor)

# Suffix

## Predict

In [ ]:
sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2 =  getSuffix(2, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t)

In [ ]:
len(sentences_2), len(next_chars_2)

In [ ]:
X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)

In [ ]:
pred = model.predict(X_test, verbose=0)
pred_a = pred[0]
pred_t = pred[1]

In [ ]:
pred[0].shape, pred[1].shape

## Evaluate

### Activity

In [ ]:
evalAct(next_chars_2, pred[0], targetchartoindice)

### Time

In [ ]:
evalTime(next_chars_t_2, pred_t, divisor)